In [2]:
import torchvision
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

c:\Users\tilak\OneDrive\Desktop\ObjectDetection\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tilak\OneDrive\Desktop\ObjectDetection\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
import random
from PIL import Image,ImageDraw
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np 
import pandas as pd
import os

import torch
import torchvision
from torchvision import transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [5]:
train=pd.read_csv("subset.csv")

In [6]:
train.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,004fd1f0df55b7fa,activemil,/m/0fly7,1,0.222326,0.485929,0.710625,0.995000,0,0,0,0,0
1,004fd1f0df55b7fa,xclick,/m/0fly7,1,0.585366,0.823640,0.625625,0.999375,1,1,0,0,0
2,08cb327bc3490925,activemil,/m/0fly7,1,0.341832,0.749764,0.812500,0.957500,0,0,0,0,0
3,09326752cf790d5d,activemil,/m/0fly7,1,0.100375,0.338649,0.385625,0.661875,1,0,0,0,0
4,09326752cf790d5d,xclick,/m/0fly7,1,0.057223,0.123827,0.296875,0.388750,0,0,0,0,0


In [7]:
df = train[["ImageID", "XMin", "YMin", "XMax", "YMax"]]

In [8]:
train.head()

,ImageID,Source,LabelName,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside
0,004fd1f0df55b7fa,activemil,/m/0fly7,1,0.222326,0.485929,0.710625,0.995000,0,0,0,0,0
1,004fd1f0df55b7fa,xclick,/m/0fly7,1,0.585366,0.823640,0.625625,0.999375,1,1,0,0,0
2,08cb327bc3490925,activemil,/m/0fly7,1,0.341832,0.749764,0.812500,0.957500,0,0,0,0,0
3,09326752cf790d5d,activemil,/m/0fly7,1,0.100375,0.338649,0.385625,0.661875,1,0,0,0,0
4,09326752cf790d5d,xclick,/m/0fly7,1,0.057223,0.123827,0.296875,0.388750,0,0,0,0,0


In [9]:
df["XMin"]=pd.to_numeric(df["XMin"])
df["YMin"]=pd.to_numeric(df["YMin"])
df["XMax"]=pd.to_numeric(df["XMax"])
df["YMax"]=pd.to_numeric(df["YMax"])

C:\Users\tilak\AppData\Local\Temp\ipykernel_988\2215214666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["XMin"]=pd.to_numeric(df["XMin"])
C:\Users\tilak\AppData\Local\Temp\ipykernel_988\2215214666.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["YMin"]=pd.to_numeric(df["YMin"])
C:\Users\tilak\AppData\Local\Temp\ipykernel_988\2215214666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [10]:
df.head()

,ImageID,XMin,YMin,XMax,YMax
0,004fd1f0df55b7fa,0.222326,0.710625,0.485929,0.995000
1,004fd1f0df55b7fa,0.585366,0.625625,0.823640,0.999375
2,08cb327bc3490925,0.341832,0.812500,0.749764,0.957500
3,09326752cf790d5d,0.100375,0.385625,0.338649,0.661875
4,09326752cf790d5d,0.057223,0.296875,0.123827,0.388750


In [11]:
unique_images=df.ImageID.unique()

In [12]:
class CustDat(torch.utils.data.Dataset):
    def __init__(self,df,unique_images,indices):
        self.df=df
        self.unique_images=unique_images
        self.indices=indices
    def __len__(self):
        return len(self.indices)
    def __getitem__(self,idx):
        image_name=self.unique_images[self.indices[idx]]
        boxes=self.df[self.df.ImageID==image_name].values[:,1:].astype("float")
        img=Image.open("object-detection\\OIDv4_ToolKit\\OID\Dataset\\train\\Jeans\\"+image_name+".jpg").convert("RGB")
        labels=torch.ones((boxes.shape[0]),dtype=torch.int64)
        target={}
        target["boxes"]=torch.tensor(boxes)
        target["label"]=labels
        return T.ToTensor()(img) ,target
        

In [13]:
train_inds ,val_inds=train_test_split(range(unique_images.shape[0]),test_size=0.1)

In [14]:
def custom_collate(data):
    return data


In [15]:
train_dl=torch.utils.data.DataLoader(CustDat(df,unique_images,train_inds),batch_size=16,shuffle=True,collate_fn=custom_collate,
                                     pin_memory=True if torch.cuda.is_available() else False)

In [16]:
val_dl=torch.utils.data.DataLoader(CustDat(df,unique_images,val_inds),batch_size=8,shuffle=True,collate_fn=custom_collate,
                                     pin_memory=True if torch.cuda.is_available() else False)

In [17]:
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes=2
in_features=model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor=FastRCNNPredictor(in_features,num_classes)

c:\Users\tilak\OneDrive\Desktop\ObjectDetection\myenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tilak\OneDrive\Desktop\ObjectDetection\myenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [19]:
optimizer=torch.optim.SGD(model.parameters(),lr=0.001,momentum=0.9,weight_decay=0.0005)
num_epochs=5

In [ ]:
model.to(device)
for epochs in range(num_epochs):
    epoch_loss=0
    for data in train_dl:
        imgs=[]
        targets=[]
        for d in data:
            imgs.append(d[0].to(device))
            targ={}
            targ["boxes"]=d[1]["boxes"].to(device)
            targ["labels"]=d[1]["label"].to(device)
            targets.append(targ)
        loss_dict=model(imgs,targets)
        loss=sum(v for v in loss_dict.values())
        epoch_loss+=loss.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch_loss)
        

In [ ]:
model.eval()
data = iter(val_dl).__next__()
img=data[0][0]
boxes=data[0][1]["boxes"]
labels=data[0][1]["label"]

In [ ]:
output=model([img.to(device)])
out_bbox=output[0]["boxes"]
out_scores=output[0]["scores"]